# HOUSE SALE PRICE ESTIMATOR - GROUP 23

## 1. Install pyspark

In [75]:
!pip install pyspark

## 2. Initialize Spark and Other required components

In [76]:
import numpy as np
import pandas as pd

from pyspark.sql import SparkSession, functions as F, DataFrame
from pyspark.ml.feature import StringIndexer, VectorAssembler, Imputer, VectorIndexer, Bucketizer, OneHotEncoder
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, GeneralizedLinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark.sql.functions import col,isnan,when,count

spark = SparkSession.builder.getOrCreate()
#spark = SparkSession.builder.master("spark://172.31.21.121:7077").getOrCreate()
spark


In [77]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## 3. Load Data

In [78]:
df_train = spark.read.csv('../data/house-sales-train.csv', inferSchema=True, header=True)
df_test = spark.read.csv('../data/house-sales-test.csv', inferSchema=True, header=True)
df_sample_submission = spark.read.csv('../data/sample_submission.csv', inferSchema=True, header=True)
col_sample_submission = ['Id','SalePrice']
df_train.cache()

DataFrame[Id: int, MSSubClass: int, MSZoning: string, LotFrontage: string, LotArea: int, Street: string, Alley: string, LotShape: string, LandContour: string, Utilities: string, LotConfig: string, LandSlope: string, Neighborhood: string, Condition1: string, Condition2: string, BldgType: string, HouseStyle: string, OverallQual: int, OverallCond: int, YearBuilt: int, YearRemodAdd: int, RoofStyle: string, RoofMatl: string, Exterior1st: string, Exterior2nd: string, MasVnrType: string, MasVnrArea: string, ExterQual: string, ExterCond: string, Foundation: string, BsmtQual: string, BsmtCond: string, BsmtExposure: string, BsmtFinType1: string, BsmtFinSF1: int, BsmtFinType2: string, BsmtFinSF2: int, BsmtUnfSF: int, TotalBsmtSF: int, Heating: string, HeatingQC: string, CentralAir: string, Electrical: string, 1stFlrSF: int, 2ndFlrSF: int, LowQualFinSF: int, GrLivArea: int, BsmtFullBath: int, BsmtHalfBath: int, FullBath: int, HalfBath: int, BedroomAbvGr: int, KitchenAbvGr: int, KitchenQual: string

In [79]:
df_train.show(n=5, truncate=False, vertical=True)

-RECORD 0----------------
 Id            | 1       
 MSSubClass    | 60      
 MSZoning      | RL      
 LotFrontage   | 65      
 LotArea       | 8450    
 Street        | Pave    
 Alley         | NA      
 LotShape      | Reg     
 LandContour   | Lvl     
 Utilities     | AllPub  
 LotConfig     | Inside  
 LandSlope     | Gtl     
 Neighborhood  | CollgCr 
 Condition1    | Norm    
 Condition2    | Norm    
 BldgType      | 1Fam    
 HouseStyle    | 2Story  
 OverallQual   | 7       
 OverallCond   | 5       
 YearBuilt     | 2003    
 YearRemodAdd  | 2003    
 RoofStyle     | Gable   
 RoofMatl      | CompShg 
 Exterior1st   | VinylSd 
 Exterior2nd   | VinylSd 
 MasVnrType    | BrkFace 
 MasVnrArea    | 196     
 ExterQual     | Gd      
 ExterCond     | TA      
 Foundation    | PConc   
 BsmtQual      | Gd      
 BsmtCond      | TA      
 BsmtExposure  | No      
 BsmtFinType1  | GLQ     
 BsmtFinSF1    | 706     
 BsmtFinType2  | Unf     
 BsmtFinSF2    | 0       
 BsmtUnfSF  

In [80]:
df_train.count()

1460

In [81]:
df_train.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- MSSubClass: integer (nullable = true)
 |-- MSZoning: string (nullable = true)
 |-- LotFrontage: string (nullable = true)
 |-- LotArea: integer (nullable = true)
 |-- Street: string (nullable = true)
 |-- Alley: string (nullable = true)
 |-- LotShape: string (nullable = true)
 |-- LandContour: string (nullable = true)
 |-- Utilities: string (nullable = true)
 |-- LotConfig: string (nullable = true)
 |-- LandSlope: string (nullable = true)
 |-- Neighborhood: string (nullable = true)
 |-- Condition1: string (nullable = true)
 |-- Condition2: string (nullable = true)
 |-- BldgType: string (nullable = true)
 |-- HouseStyle: string (nullable = true)
 |-- OverallQual: integer (nullable = true)
 |-- OverallCond: integer (nullable = true)
 |-- YearBuilt: integer (nullable = true)
 |-- YearRemodAdd: integer (nullable = true)
 |-- RoofStyle: string (nullable = true)
 |-- RoofMatl: string (nullable = true)
 |-- Exterior1st: string (nullable = true)
 |--

## 4. Feature Selection

In [82]:
def get_features (df_train):
    str_features = [] 
    int_features = []
    for col in  df_train.dtypes:
        if col[1] == 'string':
            str_features += [col[0]]
        else:
            int_features += [col[0]]
    return str_features, int_features

str_features, int_features = get_features (df_train)
print(f'str_features : {str_features}')
print(f'int_features: {int_features}')

str_features : ['MSZoning', 'LotFrontage', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
int_features: ['Id', 'MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', '

In [83]:
df_train.select(str_features).limit(5).toPandas()

,MSZoning,LotFrontage,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,GarageYrBlt,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,65,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,...,2003,RFn,TA,TA,Y,NA,NA,NA,WD,Normal
1,RL,80,Pave,NA,Reg,Lvl,AllPub,FR2,Gtl,Veenker,...,1976,RFn,TA,TA,Y,NA,NA,NA,WD,Normal
2,RL,68,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,...,2001,RFn,TA,TA,Y,NA,NA,NA,WD,Normal
3,RL,60,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,...,1998,Unf,TA,TA,Y,NA,NA,NA,WD,Abnorml
4,RL,84,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,...,2000,RFn,TA,TA,Y,NA,NA,NA,WD,Normal


In [84]:
df_train.select(int_features).limit(5).toPandas()

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,8450,7,5,2003,2003,706,0,150,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,9600,6,8,1976,1976,978,0,284,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,11250,7,5,2001,2002,486,0,434,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,9550,7,5,1915,1970,216,0,540,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,14260,8,5,2000,2000,655,0,490,...,192,84,0,0,0,0,0,12,2008,250000


#### Find count for empty, None, Null, Nan with string literals.

In [85]:
df_train.select([count(when(col(c).contains('None'), c )).alias(c) for c in df_train.columns]).show(truncate=False, vertical=True)

-RECORD 0------------
 Id            | 0   
 MSSubClass    | 0   
 MSZoning      | 0   
 LotFrontage   | 0   
 LotArea       | 0   
 Street        | 0   
 Alley         | 0   
 LotShape      | 0   
 LandContour   | 0   
 Utilities     | 0   
 LotConfig     | 0   
 LandSlope     | 0   
 Neighborhood  | 0   
 Condition1    | 0   
 Condition2    | 0   
 BldgType      | 0   
 HouseStyle    | 0   
 OverallQual   | 0   
 OverallCond   | 0   
 YearBuilt     | 0   
 YearRemodAdd  | 0   
 RoofStyle     | 0   
 RoofMatl      | 0   
 Exterior1st   | 0   
 Exterior2nd   | 0   
 MasVnrType    | 864 
 MasVnrArea    | 0   
 ExterQual     | 0   
 ExterCond     | 0   
 Foundation    | 0   
 BsmtQual      | 0   
 BsmtCond      | 0   
 BsmtExposure  | 0   
 BsmtFinType1  | 0   
 BsmtFinSF1    | 0   
 BsmtFinType2  | 0   
 BsmtFinSF2    | 0   
 BsmtUnfSF     | 0   
 TotalBsmtSF   | 0   
 Heating       | 0   
 HeatingQC     | 0   
 CentralAir    | 0   
 Electrical    | 0   
 1stFlrSF      | 0   
 2ndFlrSF 

In [86]:
df_train.select([count(when(col(c).contains('NULL'), c )).alias(c) for c in df_train.columns]).show(truncate=False, vertical=True)

-RECORD 0------------
 Id            | 0   
 MSSubClass    | 0   
 MSZoning      | 0   
 LotFrontage   | 0   
 LotArea       | 0   
 Street        | 0   
 Alley         | 0   
 LotShape      | 0   
 LandContour   | 0   
 Utilities     | 0   
 LotConfig     | 0   
 LandSlope     | 0   
 Neighborhood  | 0   
 Condition1    | 0   
 Condition2    | 0   
 BldgType      | 0   
 HouseStyle    | 0   
 OverallQual   | 0   
 OverallCond   | 0   
 YearBuilt     | 0   
 YearRemodAdd  | 0   
 RoofStyle     | 0   
 RoofMatl      | 0   
 Exterior1st   | 0   
 Exterior2nd   | 0   
 MasVnrType    | 0   
 MasVnrArea    | 0   
 ExterQual     | 0   
 ExterCond     | 0   
 Foundation    | 0   
 BsmtQual      | 0   
 BsmtCond      | 0   
 BsmtExposure  | 0   
 BsmtFinType1  | 0   
 BsmtFinSF1    | 0   
 BsmtFinType2  | 0   
 BsmtFinSF2    | 0   
 BsmtUnfSF     | 0   
 TotalBsmtSF   | 0   
 Heating       | 0   
 HeatingQC     | 0   
 CentralAir    | 0   
 Electrical    | 0   
 1stFlrSF      | 0   
 2ndFlrSF 

In [90]:
df_train.select([count(when(col(c) == '', c )).alias(c) for c in df_train.columns]).show(truncate=False, vertical=True)

-RECORD 0------------
 Id            | 0   
 MSSubClass    | 0   
 MSZoning      | 0   
 LotFrontage   | 0   
 LotArea       | 0   
 Street        | 0   
 Alley         | 0   
 LotShape      | 0   
 LandContour   | 0   
 Utilities     | 0   
 LotConfig     | 0   
 LandSlope     | 0   
 Neighborhood  | 0   
 Condition1    | 0   
 Condition2    | 0   
 BldgType      | 0   
 HouseStyle    | 0   
 OverallQual   | 0   
 OverallCond   | 0   
 YearBuilt     | 0   
 YearRemodAdd  | 0   
 RoofStyle     | 0   
 RoofMatl      | 0   
 Exterior1st   | 0   
 Exterior2nd   | 0   
 MasVnrType    | 0   
 MasVnrArea    | 0   
 ExterQual     | 0   
 ExterCond     | 0   
 Foundation    | 0   
 BsmtQual      | 0   
 BsmtCond      | 0   
 BsmtExposure  | 0   
 BsmtFinType1  | 0   
 BsmtFinSF1    | 0   
 BsmtFinType2  | 0   
 BsmtFinSF2    | 0   
 BsmtUnfSF     | 0   
 TotalBsmtSF   | 0   
 Heating       | 0   
 HeatingQC     | 0   
 CentralAir    | 0   
 Electrical    | 0   
 1stFlrSF      | 0   
 2ndFlrSF 

In [73]:
df_train.select([count(when(col(c).isNull(), c )).alias(c) for c in df_train.columns]).show(truncate=False, vertical=True)

TypeError: 'tuple' object is not callable

In [16]:
df_train.select([count(when(isnan(c), c )).alias(c) for c in df_train.columns]).show(truncate=False, vertical=True)

-RECORD 0------------
 Id            | 0   
 MSSubClass    | 0   
 MSZoning      | 0   
 LotFrontage   | 0   
 LotArea       | 0   
 Street        | 0   
 Alley         | 0   
 LotShape      | 0   
 LandContour   | 0   
 Utilities     | 0   
 LotConfig     | 0   
 LandSlope     | 0   
 Neighborhood  | 0   
 Condition1    | 0   
 Condition2    | 0   
 BldgType      | 0   
 HouseStyle    | 0   
 OverallQual   | 0   
 OverallCond   | 0   
 YearBuilt     | 0   
 YearRemodAdd  | 0   
 RoofStyle     | 0   
 RoofMatl      | 0   
 Exterior1st   | 0   
 Exterior2nd   | 0   
 MasVnrType    | 0   
 MasVnrArea    | 0   
 ExterQual     | 0   
 ExterCond     | 0   
 Foundation    | 0   
 BsmtQual      | 0   
 BsmtCond      | 0   
 BsmtExposure  | 0   
 BsmtFinType1  | 0   
 BsmtFinSF1    | 0   
 BsmtFinType2  | 0   
 BsmtFinSF2    | 0   
 BsmtUnfSF     | 0   
 TotalBsmtSF   | 0   
 Heating       | 0   
 HeatingQC     | 0   
 CentralAir    | 0   
 Electrical    | 0   
 1stFlrSF      | 0   
 2ndFlrSF 

In [17]:
df_train.select([count(when(isnan(c), c )).alias(c) for c in df_train.columns]).show(truncate=False, vertical=True)

-RECORD 0------------
 Id            | 0   
 MSSubClass    | 0   
 MSZoning      | 0   
 LotFrontage   | 0   
 LotArea       | 0   
 Street        | 0   
 Alley         | 0   
 LotShape      | 0   
 LandContour   | 0   
 Utilities     | 0   
 LotConfig     | 0   
 LandSlope     | 0   
 Neighborhood  | 0   
 Condition1    | 0   
 Condition2    | 0   
 BldgType      | 0   
 HouseStyle    | 0   
 OverallQual   | 0   
 OverallCond   | 0   
 YearBuilt     | 0   
 YearRemodAdd  | 0   
 RoofStyle     | 0   
 RoofMatl      | 0   
 Exterior1st   | 0   
 Exterior2nd   | 0   
 MasVnrType    | 0   
 MasVnrArea    | 0   
 ExterQual     | 0   
 ExterCond     | 0   
 Foundation    | 0   
 BsmtQual      | 0   
 BsmtCond      | 0   
 BsmtExposure  | 0   
 BsmtFinType1  | 0   
 BsmtFinSF1    | 0   
 BsmtFinType2  | 0   
 BsmtFinSF2    | 0   
 BsmtUnfSF     | 0   
 TotalBsmtSF   | 0   
 Heating       | 0   
 HeatingQC     | 0   
 CentralAir    | 0   
 Electrical    | 0   
 1stFlrSF      | 0   
 2ndFlrSF 

As can be seen about except for *MasVnrType* there are no columns where are empty or missing records. There are no nan records as well.

In [18]:
df_train.select( "LotFrontage" ).distinct().collect()

[Row(LotFrontage='51'),
 Row(LotFrontage='124'),
 Row(LotFrontage='54'),
 Row(LotFrontage='101'),
 Row(LotFrontage='138'),
 Row(LotFrontage='69'),
 Row(LotFrontage='112'),
 Row(LotFrontage='42'),
 Row(LotFrontage='73'),
 Row(LotFrontage='87'),
 Row(LotFrontage='64'),
 Row(LotFrontage='30'),
 Row(LotFrontage='34'),
 Row(LotFrontage='59'),
 Row(LotFrontage='160'),
 Row(LotFrontage='85'),
 Row(LotFrontage='52'),
 Row(LotFrontage='35'),
 Row(LotFrontage='NA'),
 Row(LotFrontage='71'),
 Row(LotFrontage='98'),
 Row(LotFrontage='47'),
 Row(LotFrontage='99'),
 Row(LotFrontage='110'),
 Row(LotFrontage='107'),
 Row(LotFrontage='96'),
 Row(LotFrontage='43'),
 Row(LotFrontage='100'),
 Row(LotFrontage='70'),
 Row(LotFrontage='174'),
 Row(LotFrontage='168'),
 Row(LotFrontage='61'),
 Row(LotFrontage='75'),
 Row(LotFrontage='140'),
 Row(LotFrontage='120'),
 Row(LotFrontage='46'),
 Row(LotFrontage='130'),
 Row(LotFrontage='78'),
 Row(LotFrontage='89'),
 Row(LotFrontage='77'),
 Row(LotFrontage='118'),
 R

In [19]:
def cast_to_int(_sdf: DataFrame,col_list: list) -> DataFrame:
    for col in col_list:
        _sdf = _sdf.withColumn(col, _sdf[col].cast('int'))
    return _sdf

df_test_typecast = cast_to_int(df_train, str_features)

In [20]:
str_features, int_features = get_features (df_test_typecast)
print(f'str_features : {str_features}')
print(f'int_features: {int_features}')

str_features : []
int_features: ['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'Sc

In [21]:
df_test_typecast.select(int_features).limit(5).toPandas().T

,0,1,2,3,4
Id,1.0,2.0,3.0,4.0,5.0
MSSubClass,60.0,20.0,60.0,70.0,60.0
MSZoning,NaN,NaN,NaN,NaN,NaN
LotFrontage,65.0,80.0,68.0,60.0,84.0
LotArea,8450.0,9600.0,11250.0,9550.0,14260.0
...,...,...,...,...,...
MoSold,2.0,5.0,9.0,2.0,12.0
YrSold,2008.0,2007.0,2008.0,2006.0,2008.0
SaleType,NaN,NaN,NaN,NaN,NaN
SaleCondition,NaN,NaN,NaN,NaN,NaN


## 5. EDA

There are 1460 instances of training data and 1460 of test data. Total number of attributes equals 81, of which 36 is quantitative, 43 categorical + Id and SalePrice.

##### Quantitative
1stFlrSF, 2ndFlrSF, 3SsnPorch, BedroomAbvGr, BsmtFinSF1, BsmtFinSF2, BsmtFullBath, BsmtHalfBath, BsmtUnfSF, EnclosedPorch, Fireplaces, FullBath, GarageArea, GarageCars, GarageYrBlt, GrLivArea, HalfBath, KitchenAbvGr, LotArea, LotFrontage, LowQualFinSF, MSSubClass, MasVnrArea, MiscVal, MoSold, OpenPorchSF, OverallCond, OverallQual, PoolArea, ScreenPorch, TotRmsAbvGrd, TotalBsmtSF, WoodDeckSF, YearBuilt, YearRemodAdd, YrSold

##### Qualitative
Alley, BldgType, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, BsmtQual, CentralAir, Condition1, Condition2, Electrical, ExterCond, ExterQual, Exterior1st, Exterior2nd, Fence, FireplaceQu, Foundation, Functional, GarageCond, GarageFinish, GarageQual, GarageType, Heating, HeatingQC, HouseStyle, KitchenQual, LandContour, LandSlope, LotConfig, LotShape, MSZoning, MasVnrType, MiscFeature, Neighborhood, PavedDrive, PoolQC, RoofMatl, RoofStyle, SaleCondition, SaleType, Street, Utilities,



In [22]:
saleprice_cov = {} 
for col in  df_train.dtypes:
    if col[0] != 'SalePrice' and col[1] != 'string':
        saleprice_cov[col[0]] = df_train.cov('SalePrice', col[0])

dict(sorted(saleprice_cov.items(), key=lambda item: item[1]))

{'MiscVal': -835150.301484408,
 'Id': -734073.2179575008,
 'EnclosedPorch': -624304.8923066083,
 'MSSubClass': -283233.7372285389,
 'BsmtFinSF2': -145817.53057169958,
 'LowQualFinSF': -98909.80434055999,
 'OverallCond': -6882.738552395615,
 'YrSold': -3051.540990733885,
 'KitchenAbvGr': -2378.952241636698,
 'BsmtHalfBath': -319.48489019500977,
 'BsmtFullBath': 9362.786396199308,
 'MoSold': 9972.849164843636,
 'BedroomAbvGr': 10901.465523392808,
 'HalfBath': 11350.236118752762,
 'Fireplaces': 23913.252006910352,
 'FullBath': 24538.08320110413,
 'GarageCars': 38020.18135897172,
 'TotRmsAbvGrd': 68917.16854479058,
 'OverallQual': 86904.12583679946,
 '3SsnPorch': 103837.23430948195,
 'PoolArea': 294932.3392415523,
 'ScreenPorch': 493653.54802313424,
 'YearRemodAdd': 831707.8529091986,
 'YearBuilt': 1254636.8755358825,
 'OpenPorchSF': 1662523.3606316943,
 'WoodDeckSF': 3230258.4818828786,
 'BsmtUnfSF': 7528863.6602073265,
 'GarageArea': 10589102.52274217,
 '2ndFlrSF': 11074148.695788065,
 '

We can see that LotArea, GrLivArea, TotalBsmtSF, 1stFlrSF, BsmtFinSF1 are the top 5 influencer of the Sale Price

## 6. Data Manipulation

In [39]:
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

# One Hot Encoding and nan transformation
data = pd.get_dummies(df_train)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
data = imp.fit_transform(df_test_typecast.toPandas())

# Log transformation
data = np.log(df_test_typecast.toPandas())
labels = np.log(data.SalePrice)

# Change -inf to 0 again
data[data==-np.inf]=0

In [40]:
pca = PCA(whiten=True)
pca.fit(data)
variance = pd.DataFrame(pca.explained_variance_ratio_)
np.cumsum(pca.explained_variance_ratio_)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

## 7. Transform categorical data

#### Encode a string column of labels to a column of label indices

In [49]:
for col in  df_train.dtypes:
    if col[1] != 'string':
        output_col = "" + col[0] + "_int"
        print(col[0], " -- ", output_col)
        indexer = StringIndexer(inputCol=col[0], outputCol=output_col)
        indexed = indexer.fit(df_train).transform(df_train)

Id  --  Id_int
MSSubClass  --  MSSubClass_int
LotArea  --  LotArea_int
OverallQual  --  OverallQual_int
OverallCond  --  OverallCond_int
YearBuilt  --  YearBuilt_int
YearRemodAdd  --  YearRemodAdd_int
BsmtFinSF1  --  BsmtFinSF1_int
BsmtFinSF2  --  BsmtFinSF2_int
BsmtUnfSF  --  BsmtUnfSF_int
TotalBsmtSF  --  TotalBsmtSF_int
1stFlrSF  --  1stFlrSF_int
2ndFlrSF  --  2ndFlrSF_int
LowQualFinSF  --  LowQualFinSF_int
GrLivArea  --  GrLivArea_int
BsmtFullBath  --  BsmtFullBath_int
BsmtHalfBath  --  BsmtHalfBath_int
FullBath  --  FullBath_int
HalfBath  --  HalfBath_int
BedroomAbvGr  --  BedroomAbvGr_int
KitchenAbvGr  --  KitchenAbvGr_int
TotRmsAbvGrd  --  TotRmsAbvGrd_int
Fireplaces  --  Fireplaces_int
GarageCars  --  GarageCars_int
GarageArea  --  GarageArea_int
WoodDeckSF  --  WoodDeckSF_int
OpenPorchSF  --  OpenPorchSF_int
EnclosedPorch  --  EnclosedPorch_int
3SsnPorch  --  3SsnPorch_int
ScreenPorch  --  ScreenPorch_int
PoolArea  --  PoolArea_int
MiscVal  --  MiscVal_int
MoSold  --  MoSold_i

#### Assembler combines all integer and create a vector which is used as input to predict.

In [53]:
str_features, int_features = get_features (df_train)
assembler= VectorAssembler(inputCols=int_features,outputCol="features")

In [54]:
output= assembler.transform(indexed)
output.select("features","SalePrice")

DataFrame[features: vector, SalePrice: int]

In [55]:
#We can see column features is dense vector
final=output.select("features","SalePrice")
final.head(3)

[Row(features=DenseVector([1.0, 60.0, 8450.0, 7.0, 5.0, 2003.0, 2003.0, 706.0, 0.0, 150.0, 856.0, 856.0, 854.0, 0.0, 1710.0, 1.0, 0.0, 2.0, 1.0, 3.0, 1.0, 8.0, 0.0, 2.0, 548.0, 0.0, 61.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2008.0, 208500.0]), SalePrice=208500),
 Row(features=DenseVector([2.0, 20.0, 9600.0, 6.0, 8.0, 1976.0, 1976.0, 978.0, 0.0, 284.0, 1262.0, 1262.0, 0.0, 0.0, 1262.0, 0.0, 1.0, 2.0, 0.0, 3.0, 1.0, 6.0, 1.0, 2.0, 460.0, 298.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 2007.0, 181500.0]), SalePrice=181500),
 Row(features=DenseVector([3.0, 60.0, 11250.0, 7.0, 5.0, 2001.0, 2002.0, 486.0, 0.0, 434.0, 920.0, 920.0, 866.0, 0.0, 1786.0, 1.0, 0.0, 2.0, 1.0, 3.0, 1.0, 6.0, 1.0, 2.0, 608.0, 0.0, 42.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.0, 2008.0, 223500.0]), SalePrice=223500)]

In [56]:
#We will split data into train and validate
train_df,valid_df= final.randomSplit([0.7,0.3])
train_df.describe().show()

+-------+------------------+
|summary|         SalePrice|
+-------+------------------+
|  count|               990|
|   mean|181310.71616161618|
| stddev| 79134.44178927108|
|    min|             34900|
|    max|            625000|
+-------+------------------+



In [57]:
#initializing and fitting model
lr= LinearRegression(labelCol="SalePrice")
model= lr.fit(train_df)

In [58]:
#fitting model of validation set
validate=model.evaluate(valid_df)

In [59]:
#let's check how model performed
print(validate.rootMeanSquaredError)
print(validate.r2)

0.032957663453155095
0.9999999999998306


## 8. Build Pipeline

In [60]:
_stages = []

##Imputer
null_impute = Imputer(inputCols= int_features, outputCols=int_features) 
_stages += [null_impute]

##Encoder
str_indexer = [StringIndexer(inputCol=column,
                           outputCol=f'{column}_StringIndexer',
                            handleInvalid='keep') 
               for column in str_features]
_stages += str_indexer

#Assembler
assembler_input = [f for f in int_features] 
assembler_input += [f'{column}_StringIndexer' 
                    for column  in str_features] 
feature_vector = VectorAssembler(inputCols=assembler_input, 
                                 outputCol='features', 
                                 handleInvalid = 'keep' )
_stages += [feature_vector]

#Vector Encoder
vect_indexer = VectorIndexer(inputCol='features', 
                             outputCol= 'features_indexed', 
                             handleInvalid = 'keep' )
_stages += [vect_indexer]

#Model
LR = LinearRegression(featuresCol='features_indexed', 
                      labelCol= 'SalePrice',
                     maxIter=10,
                     regParam=0.3,
                     elasticNetParam=0.8)
_stages += [LR]

In [62]:
ml_pipeline = Pipeline(stages=_stages)
model = ml_pipeline.fit(df_train)

In [63]:
df_predict = model.transform(df_test)

IllegalArgumentException: requirement failed: Column BsmtFinSF1 must be of type numeric but was actually of type string.